In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# ---------- load data ----------
df = pd.read_csv("../DataSets/wellbeing_long_with_domain.csv")

# make sure these column names match your file
# print(df.columns)

countries = sorted(df["Reference area"].unique())
years = sorted(df["TIME_PERIOD"].unique())

# map each Domain to a real color
domain_colors = {
    "Income & wealth":        "#1f77b4",
    "Work & job quality":     "#ff7f0e",
    "Housing":                "#2ca02c",
    "Health":                 "#d62728",
    "Knowledge & skills":     "#9467bd",
    "Social connections":     "#8c564b",
    "Safety":                 "#e377c2",
    "Subjective well-being":  "#7f7f7f",
}

# ---------- function to build one trace ----------
def make_trace(country, year):
    d = df[(df["Reference area"] == country) &
           (df["TIME_PERIOD"] == year)].copy()

    # order so same-domain variables are together
    d = d.sort_values(["Domain", "Metric"]).reset_index(drop=True)

    # categorical angular labels = metric names
    d["theta_label"] = d["Metric"]
    d["Radius"] = d["Value"]

    # map domains to colors
    colors = d["Domain"].map(domain_colors).tolist()

    return go.Barpolar(
        r=d["Radius"],
        theta=d["theta_label"],      # category labels, not degrees
        marker=dict(
            color=colors,
            line=dict(color="white", width=1)
        ),
        hovertext=d["Metric"],
        hovertemplate="%{hovertext}<br>Value=%{r:.3f}<extra></extra>",
    )

# ---------- initial country ----------
init_country = countries[0]

# one trace per year for the initial country (slider controls visibility)
traces = [make_trace(init_country, y) for y in years]
for i, tr in enumerate(traces):
    tr.visible = (i == 0)

fig = go.Figure(data=traces)

# ---------- slider (years) ----------
steps = []
for i, y in enumerate(years):
    step = dict(
        method="update",
        args=[{"visible": [j == i for j in range(len(years))]},
              {"title": f"{init_country} – {y}"}],
        label=str(y),
    )
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    pad={"t": 40},
    steps=steps
)]

# ---------- country dropdown ----------
buttons = []
for c in countries:
    data_c = [make_trace(c, y) for y in years]
    for i, tr in enumerate(data_c):
        tr.visible = (i == 0)
    buttons.append(
        dict(
            label=c,
            method="update",
            args=[
                {"data": data_c,
                 "visible": [i == 0 for i in range(len(years))]},
                {"title": f"{c} – {years[0]}",
                 "sliders": sliders}
            ],
        )
    )

# ---------- layout ----------
fig.update_layout(
    title=f"{init_country} – {years[0]}",
    template="plotly_white",
    polar=dict(
        angularaxis=dict(
            type="category",          # use metric names
            direction="clockwise",
            rotation=90               # optional: where to start (deg)
        ),
        radialaxis=dict(
            range=[0, 1],
            ticks="outside"
        ),
    ),
    legend_title_text="Domain",
    sliders=sliders,
    updatemenus=[dict(
        type="dropdown",
        x=0.0, y=1.15,
        showactive=True,
        buttons=buttons
    )]
)

fig.show()
